In [1]:
# Import the NumPy library for numerical computations and array operations
import numpy as np


In [2]:
def mod_inverse(a, m):
    """Finds the modular inverse of a under modulo m using brute force."""

    # Ensure 'a' is within the modulo range
    a = a % m

    # Iterate through all possible values from 1 to m-1
    for x in range(1, m):
        # Check if (a * x) % m == 1, meaning x is the modular inverse of a
        if (a * x) % m == 1:
            return x  # Return the modular inverse

    return None  # If no modular inverse exists, return None


In [3]:
def hill_cipher_encrypt(plaintext, key_matrix):
    """Encrypts plaintext using the Hill Cipher."""

    n = len(key_matrix)  # Determine the size of the key matrix (n x n)

    # Convert plaintext to uppercase and remove spaces
    plaintext = plaintext.upper().replace(" ", "")

    # Padding if the plaintext length is not a multiple of n
    if len(plaintext) % n != 0:
        plaintext += "X" * (n - len(plaintext) % n)  # Append 'X' as padding

    # Convert plaintext characters to numerical representation (A=0, B=1, ..., Z=25)
    plaintext_vector = [ord(char) - ord('A') for char in plaintext]

    ciphertext = ""  # Initialize ciphertext string

    # Process the plaintext in blocks of size 'n'
    for i in range(0, len(plaintext_vector), n):
        block = plaintext_vector[i:i + n]  # Extract a block of n elements

        # Multiply key matrix with the plaintext block and apply modulo 26
        result = np.dot(key_matrix, block) % 26

        # Convert the numerical result back to characters and append to ciphertext
        ciphertext += "".join(chr(num + ord('A')) for num in result)

    return ciphertext  # Return the encrypted text


In [4]:
def hill_cipher_decrypt(ciphertext, key_matrix):
    """Decrypts ciphertext using the Hill Cipher."""

    n = len(key_matrix)  # Determine the size of the key matrix (n x n)

    # Compute determinant of the key matrix and take modulo 26
    determinant = int(round(np.linalg.det(key_matrix))) % 26

    # Compute modular inverse of the determinant under mod 26
    mod_inv = mod_inverse(determinant, 26)

    # If no modular inverse exists, the key matrix is not invertible
    if mod_inv is None:
        raise ValueError("Key matrix is not invertible under mod 26")

    # Compute the adjugate (adjoint) matrix, rounded and taken modulo 26
    adjugate = np.round(determinant * np.linalg.inv(key_matrix)).astype(int) % 26

    # Compute inverse key matrix in mod 26
    inverse_key_matrix = (mod_inv * adjugate) % 26

    # Convert ciphertext characters to numerical representation (A=0, B=1, ..., Z=25)
    ciphertext_vector = [ord(char) - ord('A') for char in ciphertext]

    plaintext = ""  # Initialize decrypted plaintext string

    # Process the ciphertext in blocks of size 'n'
    for i in range(0, len(ciphertext_vector), n):
        block = ciphertext_vector[i:i + n]  # Extract a block of n elements

        # Multiply inverse key matrix with the ciphertext block and apply modulo 26
        result = np.dot(inverse_key_matrix, block) % 26

        # Convert the numerical result back to characters and append to plaintext
        plaintext += "".join(chr(num + ord('A')) for num in result)

    return plaintext  # Return the decrypted text


In [6]:
# Example usage of Hill Cipher

plaintext = "HELLO"  # Input plaintext message
key_matrix = np.array([[6, 24, 1], [13, 16, 10], [20, 17, 15]])  # Example 3x3 key matrix

# Ensure the plaintext length is a multiple of the matrix size (3x3 in this case)
if len(plaintext) % key_matrix.shape[0] != 0:
    plaintext += "X" * (key_matrix.shape[0] - len(plaintext) % key_matrix.shape[0])
    # If not, pad with 'X' to make the length a multiple of 3

# Encrypt the plaintext using the Hill Cipher encryption function
encrypted_text = hill_cipher_encrypt(plaintext, key_matrix)
print("Encrypted:", encrypted_text)  # Display the encrypted text

# Decrypt the encrypted text back to the original plaintext
print("Decrypted:", hill_cipher_decrypt(encrypted_text, key_matrix))  # Display the decrypted text


Encrypted: TFJJZX
Decrypted: GHNKVN
